# Learning a shortest-path proxy

## Data set definition

In [17]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.utils.graph import graph_shortest_path
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as preprocessing

n_points = 5000

#X, color = datasets.samples_generator.make_s_curve(n_points, random_state=0)
X, color = datasets.samples_generator.make_swiss_roll(n_points, random_state=0, noise=0.7)
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0., 1.))
X = min_max_scaler.fit_transform(X)

fig = plt.figure(figsize=(15, 15), dpi=150)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.Spectral)
ax.view_init(4, -72)
plt.title("Learning shortest path prediction with %i points" % (n_points), fontsize=14)
plt.show()


## Shortest-path calculation between all points

In [ ]:
n_neighbors = 10
n_components = 2

class Geodesic():

    def __init__(self, n_neighbors=5, path_method='auto',
                 neighbors_algorithm='auto', n_jobs=None):
        self.n_neighbors = n_neighbors
        self.path_method = path_method
        self.neighbors_algorithm = neighbors_algorithm
        self.n_jobs = n_jobs

    def get_distance_matrix(self, X):
        nbrs = NearestNeighbors(n_neighbors=self.n_neighbors,
                                      algorithm=self.neighbors_algorithm,
                                      n_jobs=self.n_jobs)
        nbrs.fit(X)

        kng = kneighbors_graph(nbrs, self.n_neighbors,
                               mode='distance', n_jobs=self.n_jobs)
        G = graph_shortest_path(kng,
                                 method=self.path_method,
                                 directed=False)

        return G, kng, nbrs

G, _, _ = Geodesic().get_distance_matrix(X)
G = np.abs(G)

plt.imshow(np.abs(G) / np.max(np.abs(G)))
plt.title("Distance Matrix")
plt.show()

## Data set definition for learning

In [20]:
X_set = []
Y_set = []

def get_training_sample(x_1, x_2):
    #return np.concatenate((x_1, x_2, x_1 - x_2, x_2 - x_1, np.linalg.norm(x_1 - x_2, keepdims=True)), axis=0)
    return np.concatenate((x_1, x_2), axis=0)

for x_1, idx_1 in zip(X, range(len(X))):
    for x_2, idx_2 in zip(X[idx_1:], range(idx_1, len(X))):
        X_set.append(get_training_sample(x_1, x_2))
        Y_set.append(G[idx_2, idx_1])

X_set = np.asarray(X_set)
X_set = X_set.reshape((len(X_set), np.prod(X_set.shape[1:])))
Y_set = np.asarray(Y_set)[:,np.newaxis]

# double the data set by mirroring

X_set = np.concatenate((X_set, np.concatenate((X_set[:,:3],X_set[:,3:]), axis = 1)), axis = 0)
Y_set = np.concatenate((Y_set, Y_set), axis = 0)

X_train, X_test, Y_train, Y_test = train_test_split(X_set, Y_set, test_size=0.1, random_state=0)

print("Shape (X_set, Y_set)", (X_set.shape, Y_set.shape))
print("Shape (X_train, Y_train)", (X_train.shape, Y_train.shape))
print("Shape (X_test, Y_test)", (X_test.shape, Y_test.shape))

Shape (X_set, Y_set) ((25005000, 6), (25005000, 1))
Shape (X_train, Y_train) ((22504500, 6), (22504500, 1))
Shape (X_test, Y_test) ((2500500, 6), (2500500, 1))


## Learn the proxy

In [21]:

from keras.models import Model
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam

# train the model
epochs, batch_size = 25, 1024

inputs = Input(shape=(X_set.shape[-1],))
h = Dense(64, activation='elu')(inputs)
#h = Dropout(0.5)(h)
h = Dense(64, activation='elu')(h)
#h = Dropout(0.5)(h)
#h = Dense(64, activation='elu')(h)
#h = Dropout(0.5)(h)
predictions = Dense(1, activation='relu')(h)
model = Model(inputs=inputs, outputs=predictions)

# Compile model
# mean_absolute_error, mean_absolute_percentage_error ,mean_squared_error, mean_squared_logarithmic_error, squared_hinge, hinge
model.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False))
model.fit(X_train, Y_train, verbose=2, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test))



Train on 22504500 samples, validate on 2500500 samples
Epoch 1/25
 - 69s - loss: 0.3024 - val_loss: 0.1760
Epoch 2/25
 - 72s - loss: 0.1599 - val_loss: 0.1597
Epoch 3/25
 - 79s - loss: 0.1479 - val_loss: 0.1431
Epoch 4/25
 - 83s - loss: 0.1412 - val_loss: 0.1383
Epoch 5/25
 - 86s - loss: 0.1368 - val_loss: 0.1338
Epoch 6/25
 - 87s - loss: 0.1328 - val_loss: 0.1371
Epoch 7/25
 - 145s - loss: 0.1287 - val_loss: 0.1338
Epoch 8/25
 - 146s - loss: 0.1248 - val_loss: 0.1253
Epoch 9/25
 - 131s - loss: 0.1206 - val_loss: 0.1236
Epoch 10/25
 - 168s - loss: 0.1162 - val_loss: 0.1368
Epoch 11/25
 - 166s - loss: 0.1120 - val_loss: 0.1200
Epoch 12/25
 - 180s - loss: 0.1079 - val_loss: 0.1081
Epoch 13/25
 - 119s - loss: 0.1041 - val_loss: 0.1096
Epoch 14/25
 - 129s - loss: 0.1006 - val_loss: 0.1082
Epoch 15/25


KeyboardInterrupt: 

## Show the distribution of predicted distances

In [ ]:
Y_pred = model.predict(X_test)
min_max = (np.concatenate((Y_test,Y_pred)).min(), np.concatenate((Y_test,Y_pred)).max())
plt.hist(Y_test, 50, range=min_max, density=False, facecolor='g', alpha=0.75)
plt.hist(Y_pred, 50, range=min_max, density=False, facecolor='r', alpha=0.75)

plt.xlabel('path distance')
plt.ylabel('# distances')
plt.legend(["ground truth", "predicted"])
plt.title('Distribution of Distances')
plt.grid(True)
plt.show()

## Pick a point and show the predicted distance to every other point

In [ ]:
pick = 0
X_viz = []
for x in X:
    X_viz.append(get_training_sample(X[pick], x))
X_viz = np.asarray(X_viz)
X_viz = X_viz.reshape((len(X_viz), np.prod(X_viz.shape[1:])))

fig = plt.figure(figsize=(15, 15), dpi=150)
plt.suptitle("Shortest-path vs. prediction: Current pick is indicated by 'x'", fontsize=14)

ax = fig.add_subplot(121, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=G[pick]/G[pick].max(), cmap=plt.cm.jet)
ax.scatter(X[pick, 0], X[pick, 1], X[pick, 2], s=2000, linewidth=10, marker='x', c=[0.0], cmap=plt.cm.gray)
ax.view_init(4, -72)
ax.set_title("Ground-truth")
ax = fig.add_subplot(122, projection='3d')
Y_viz = model.predict(X_viz)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=np.squeeze(Y_viz/Y_viz.max()), cmap=plt.cm.jet)
ax.scatter(X[pick, 0], X[pick, 1], X[pick, 2], s=2000, linewidth=10, marker='x', c=[0.0], cmap=plt.cm.gray)
ax.view_init(4, -72)
ax.set_title("Prediction")
plt.show()

## Store the model

In [ ]:
print("Store model")
model.save('models/shortest_path_predictor_0to1.h5')